In [ ]:
#Uniquement utilse sous Colab
import sys
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    !rm -rf ESGI-M2-IABD/
    !git clone https://github.com/pcouy/ESGI-M2-IABD
    !cp -r ESGI-M2-IABD/Cours01/imgs .
    !pip install --upgrade pip
    !pip install ESGI-M2-IABD/code

# Apprentissage par renforcement - Rappels

L'apprentissage par renforcement consiste à apprendre comment choisir des actions à partir de situations, dans le but de maximiser un signal de récompense. L'agent apprenant n'a pas de consigne sur les actions à choisir. Il doit découvrir lui-même quelles actions lui permettent d'obtenir le plus de récompenses en les essayant.

## Processus de Décision Markovien (MDP)

Les MDP sont une forme mathématiquement idéalisée du problème d'apprentissqge par renforcement, sur laquelle des affirmations mathématiques précises peuvent être formulées. Le formalisme mathématique associé est conçu pour modéliser le problème de l'apprentissage par l'interraction pour atteindre un but.

* **Agent :** Décideur et apprenant dans l'*environnement*
* **Environnement :** Tout ce qui est extérieur à l'*agent*
* **État :** Peut désigner l'état de l'environnement $s_t \in \mathsf{S}$. Dans ce cas, il s'agit de toutes les variables, visibles ou cachées pour l'*agent*, qui caractérisent l'environnement. Désigne aussi fréquemment, par abus de langage, l'*état perçu par l'agent* ou *observation* $o_t \in \mathsf{O}$. Selon le problème étudié, l'état de l'environnement peut être entièrement connu de l'agent ($o_t = s_t$), ou observé de manière indirecte par l'agent ($o_t = f(s_t)$ où $f : \mathsf{S} \to \mathsf{O}$ est une fonction associant par exemple un état de l'environnement à des mesures effectuées par des capteurs). Lorsque l'état observé par l'agent contient toutes les informations ayant un impact sur le futur, on dit que l'état a la *propriété de Markov*.

![Interactions agent-environnement dans un MDP](imgs/agent_env_interaction.png)

À chaque pas de temps $t = 0,1,2,...$ , l'agent et l'environnement interragissent. L'agent reçoit une représentation de l'état de l'environnement $f(s_t) = o_t \in \mathsf{O}$, utilise cette observation pour choisir une action $a_t \in \mathsf{A}$. Un pas de temps plus tard, l'agent reçoit une récompense numérique $r_t \in \mathsf{R} \subset \mathsf{R}$ et l'environnement se trouve dans un nouvel état $s_{t+1}$ qui peut à son tour être observé par l'agent. 

L'environnement évolue selon sa fonction de transition $p(s', r | s, a)$ qui donne la probablilité que $s_{t+1} = s'$ et $r_{t+1} = r$ sachant que $s_t = s$ et $a_t = a$.

Dans la figure ci-dessus, la fonction $f$ qui génère les observations à partir de l'état de l'environnement est internalisée dans l'environnement. Dans le cas particulier où $f(s_t)=s_t$, l'environnement est totalement observé. C'est le cas par exemple des jeux à information complète. Cette fonction n'est dans la pratique pas toujours connue. Il est aussi possible de transformer volontairement les observations rendues disponibles par l'environnement afin d'en extraire les informations utiles à la prise de décision. C'est le cas, par exemple, lorsqu'on approxime la *fonction de valeur* par une combinaison linéaire de *features* expertes.

> **Exercice :** Trouver des exemples variés de tâches qui rentrent dans le cadre des MDPs, en définissant les espaces des états, des actions et des récompenses. Essayer de trouver un example qui atteint les limites du modèle.

**Répondre ici**

> **Exercice :** Considérez le problème de la conduite d'une voiture. Donnez plusieurs formulations du problème comme un MDP, correspondant à différents niveaux d'abstraction. Où faut-il placer la séparation entre l'agent et l'environnement ? Selon quels critères décidez-vous de placer cette limite ?

**Répondre ici**

## Récompense, Retours et Épisodes

La récompense est l'unique signal requis pour caractériser tous les buts de l'agent. L'utilisation de la 
récompense pour formaliser les objectifs est une des caractéristiques les plus distinctives de l'apprentissage par renforcement. Il s'agit d'une méthode ayant fait ses preuves en terme de flexibilité et de champs d'application.

| Probleme | Recompense |
| -------------- | --------------- |
| Apprentissage de la marche |  |
| Navigation dans un labyrinthe |  |
| Robot ramasseur d'ordures |  |
| Joueur d'échec |  |

Si nous voulons que l'agent réalise une tâche particulière, nous devons lui fournir un signal de récompense construit de manière à ce que sa maximisation entraine la réalisation de cette tâche. Il est critique de définir une récompense qui représente réellement ce que nous voulons voir accompli.

> **Exercice :** De quelle manière pourrions nous définir la récompense pour y inclure nos connaissances sur le "comment" de la réalisation de l'objectif. Que risque-t-il alors de se passer ?

**Répondre ici**

La récompense est le moyen de *donner son objectif à l'agent*, pas de lui expliquer *comment y arriver* !

Le but de l'agent étant de maximiser les récompenses cumulées sur le long terme, comment définir cet objectif formellement ? Si la séquence de récompenses obtenues après le pas de temps $t$ est $r_{t+1}, r_{t+2}, ...$, quelle est précisément la caractéristique de cette séquence que nous cherchons à maximiser. De manière générale, on cherche à maximiser l'*espérance de retour* (au sens statistique de l'espérance), où le retour, noté $g_t$ est une fonction de la séquence de récompenses. Dans le cas le plus simple, le retour est la somme des récompenses tel que $g_t = r_{t+1}+r_{t+2}+...+r_T$ ; où $T$ est le pas de temps final.

Cette approche fonctionne dans les situations où il existe une notion naturelle de pas de temps final. Cela correspond à dire que les séquences d'interactions agent-environnement se divisent naturellement en sous-séquences, que l'on appelle *épisodes*. Chaque épisode se termine par un état spécial, que l'on appelle état terminal, suivi d'une réinitialisation à un état de départ.

Il existe d'autre part des tâches qui ne se séparent pas en épisodes distincts, mais au contraire continuent indéfiniment. La formulation précédente du retour devient alors problématique puisque dans ces cas, $T=\infty$ et le retour (que nous cherchons à maximiser) peut devenir infini.

La définition du retour est donc modifiée de la manière suivante :

$g_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} = \sum_{k=0}^{\infty}\gamma^k r_{t+k+1}$ où $\gamma$ tq $0 \le \gamma \le 1$ est le *taux de discount*

Le taux de discount détermine la valeur présente des récompenses futures. Si $\gamma \lt 1$ alors la somme infinie converge (a une valeur finie) tant que les récompenses sont bornées. Si $\gamma=0$, l'agent est dit "myope" son choix d'actions $a_t$ visera à maximiser uniquement la récompense immédiate $r_{t+1}$.

On note que le *retour* peut être défini de façon récursive

$
g_t = r_{t+1} + \gamma r_{t+2} + \gamma^2 r_{t+3} + \gamma^3 r_{t+4} + ... \\
g_t = r_{t+1} + \gamma\left( r_{t+2} + \gamma r_{t+3} + \gamma^2 r_{t+4} + ... \right) \\
g_t = r_{t+1} + \gamma g_{t+1}
$

Cette formulation récursive est valable pour tout $t \lt T$

## Éléments de l'apprentissage par renforcement

On considère à partir de maintenant, par souci de simplicité, que les *états* et les *observations* sont la même chose.

* **Politique :** Fonction $\pi : \mathsf{O} \times \mathsf{A} \to [0;1]$  qui décrit la façon d'agir d'un agent à tout instant donné. Il s'agit d'une fonction associant une *observation* et une *action* à la probabilité de choisir l'action donnée étant donnée l'observation. On note $\pi(a|s)$ la probabilité de choisit l'action $a$ dans l'état $s$.
* **Fonction de valeur :** La *valeur* d'un état $s$ sous une politique $\pi$, notée $v_\pi(s)$ est l'espérance de retour à partir de l'état $s$ en suivant la politique $\pi$. De même, on définit la valeur d'une action $a$ dans l'état $s$ sous la politique $\pi$, notée $q_\pi(s,a)$, comme l'espéreance de retour à partir de l'état $s$, en effectuant l'action $a$ puis en suivant la politique $\pi$.

Les fonctions de valeur $v$ et $q$ peuvent être estimées à partir des expériences. Par exemple, un agent suit une politique $\pi$ et maintient, pour chaque état $s$ rencontré, une table des retours moyens suivant cet état. Dans ce cas, les moyennes convergeront vers la valeur $v_\pi(s)$ au fur et à mesure que le nombre d'occurence de cet état dans l'expérience de l'agent augmentera.

Une propriété fondamentale des fonctions de valeur est, en tant qu'estimateur du retour, de se définir récursivement, de même que le retour dans la section précédente :

$
v_\pi(s) = E_\pi[g_t | s_t=s] \\
v_\pi(s) = E_\pi[r_{t+1} + \gamma g_{t+1} | s_t=s] \\
v_\pi(s) = \sum_a \pi(a|s)\sum_{s'}\sum_{r} p(s',r|s,a)\left[r + \gamma E_\pi [g_{t+1}|s_{t+1} = s']\right] \\
v_\pi(s) = \sum_a \pi(a|s)\sum_{s',r} p(s',r|s,a)\left[r + \gamma v_\pi(s')\right]
$

Il s'agit de l'*équation de Bellman* pour $v_\pi$. Elle exprime une relation entre la valeur d'un état et les valeurs des états successeurs. La fonction de valeur $v_\pi$ est l'unique solution à l'équation de Bellman. Cette équation est la base de plusieurs méthodes permettant de calculer, approximer et apprendre $v_\pi$.

Résoudre une tâche d'apprentissage par renforcement signifie trouver une politique qui permet d'obtenir beaucoup de récompense sur le long terme. Pour les MDP finis, on peut définir précisément une politique optimale :

Les fonctions de valeur définissent un ordre partiel entre les politiques. Une politique $\pi$ est dite meilleure ou égale à une politique $\pi'$ si son espérance de retour est supérieure ou égale à celle de $\pi'$ pour tous les états $s \in \mathsf{S}$ : $\pi \ge \pi' \iff \forall s \in \mathsf{S} , v_\pi(s) \ge v_{\pi'}(s)$.

Il existe toujours au moins une politique qui est meilleure (ou égale) que toutes les autres : il s'agit de la *politique optimale*. Bien qu'il puisse en exister plus d'une, on note toutes les politiques optimales $\pi_*$. Elles partagent toutes la même fonction de valeur, appelée *fonction de valeur optimale*, notée $v_*$ et définie ainsi :

$
v_*(s) = \max_\pi v_\pi(s) , \forall s \in \mathsf{S}
$

Puisque $v_*$ est la fonction de valeur d'une politique, elle doit satisfare l'équation de Bellman. Puisque cette fonction de valeur est optimale, l'équation de Bellman peut être écrite sous une forme spéciale sans avoir à faire référence à une politique. Il s'agit de l'*équation d'optimalité de Bellman* :

$
v_*(s) = \max_{a \in \mathsf{A}(s)} q_{\pi_*}(s,a) \\
v_*(s) = \max_a E_{\pi_*}[g_t | s_t=s, a_t=a]\\
v_*(s) = \max_a E_{\pi_*}[r_{t+1} + \gamma g_{t+1} | s_t=s, a_t=a] \\
v_*(s) = \max_a E[r_{t+1} + \gamma v_*(s_{t+1}) | s_t=s,a_t=a] \\
v_*(s) = \max_a \sum_{s',r} p(s',r|s,a)[r+\gamma v_*(s')]
$

Pour les MDP finis, l'équation d'optimalité de Bellman a une unique solution. Il s'agit en réalité d'un système d'équations, une pour chaque état : pour $n$ état la formule définit un système de $n$ équations à $n$ inconnues. Si la fonction de transition $p$ est connue pour l'environnement, il devient en principe possible de résoudre le système d'équations. 

Une fois $v_*$ déterminée, il devient relativement facile de déterminer une politique optimale. Pour chaque état $s$, il y aura une ou plusieurs actions pour lesquelles on atteindra le maximum utilisé dans l'équation d'optimalité de Bellman. Toute politique qui assigne des probabilités non-nulles uniquement à ces actions est optimale.

Résoudre analytiquement l'équation d'optimalité de Bellman est une manière de trouver une politique optimale, et donc de résoudre un problème d'apprentissage par renforcement. Cette solution est toutefois rarement possible : cela revient à réaliser une recherche exhaustive dans l'arbre des possibles afin de calculer la probabilité d'occurence et la récompense attendue de chaque état. En effet, cette solution tient à 3 conditions qui sont rarement vraies en pratique :

1. La connaissance précise de la fonction de transition $p(s',r|s,a)$
2. Disposer de puissance de calcul suffisante
3. L'état observé doit avoir la propriété de Markov

De nombreuses méthodes de résolution de problèmes peuvent être formulées comme des moyens de résoudre approximativement l'équation d'optimalité de Bellman. La méthode *minimax* et sa variante *alpha-beta pruning* consistent à développer itérativement $v_*(s')$ dans le côté droit de l'équation puis à utiliser une fonction heuristique pour approximer $v_*$ au dernier niveau exploré de l'arbre.

De nombreuses méthodes d'apprentissage par renforcement consistent à résoudre approximativement l'équation d'optimalité de Bellman en utilisant les transitions vécues par l'agent à la place de connaissances *a priori* sur les transitions attendues.

Toutes les équations définies à partir de la fonction de valeur des états $v$ peuvent être définies de manière analogue avec la fonction de valeur des actions $q$.

## Exploration vs exploitation

### *k-bandits*

Le problème des *k-bandits* est souvent utilisé pour illustrer le compromis entre exploration et exploitation de la fonction de valeur.

> Un utilisateur fait face à une machine à sous disposant de k leviers, et doit décider de quels leviers actionner. Chaque levier a une distribution de gain différente, que l'utilisateur ne connait pas *a priori*.

> **Exercice :** Formuler le problème des *k-bandits* comme un MDP simple en donnant les ensembles des états, des actions, des récompenses, et la fonction de transition de ce MDP.

**Répondre ici**

Le fait de choisir l'action dont la valeur estimée est la plus grande s'appelle la politique *greedy* (gloutonne en français).

> **Exercice :** Dans quelles situations est-il souhaitable de choisir une action dont la valeur estimée n'est pas la plus grande ? Pourquoi ?

**Répondre ici**

> **Exercice :** On considère le cas où l'estimation $q(a)$ de la fonction de valeur optimale $q_*$ est le gain moyen (depuis le début du temps) par actionnage du levier $a$, initialisé à $q_0=0$. Donner des exemples de politiques simples obtenant une récompense moyenne par action plus importante que la politique *greedy*. Que se passe-t-il si $q_0$ est positif ? négatif ?

**Répondre ici**

## Mise en oeuvre - Gridworld

On utilise souvent des environnements dits *gridworld* comme exemples de MDP. Ces environnements peuvent être décrit de la manière suivante :

* Une grille de taille $X \times Y $dont chaque case représente un état du MDP : $\mathsf{S} = [1..X] \times [1..Y]$
* Les actions possibles sont $\mathsf{A} = {N,E,S,W}$, correspondant aux mouvements vers les cases adjacentes.
* Les récompenses sont déterminées en fonction de l'état $s'$ d'arrivée à chaque pas de temps. Il y a au moins un état terminal.
* La fonction de transition peut être déterministe (les mouvements réels correspondent fidèlement aux actions choisies par l'agent) ou stochastique (il y a une probabilité pour que l'action choisie par l'agent ne débouche pas sur l'état attendu). De même, les récompenses peuvent être déterministes ou stochastiques.

Nous nous intéressons ici à un environnement *gridworld* constitué de 4 types de cases :

1. Les cases libres : n'octroient pas de récompense quand l'agent arrive dessus
2. Les murs : cases infranchissables, l'agent reçoit une récompense de $-0.01$ s'il tente d'aller dessus
3. Les objectifs : état terminal ayant une récompense de $+10$
4. La lave : état terminal ayant une récompense de $-2$

L'état observé par l'agent est simplement un identifiant unique correspondant à la case sur laquelle il se trouve.

Nous utiliserons dans cette partie l'algorithme du *Q-learning* avec une fonction de valeur tabulaire : les valeurs approximées $q(s,a)$ sont maintenues dans un dictionnaire à 2 niveaux tel que $q(s,a) = Q[s][a]$

![Algorithme du Q-learning](imgs/q_learning_algo.png)

### Cas déterministe - Présentation des outils

On commence par importer et paramétrer les modules nécessaires :

In [ ]:
import code_tp as TP
import gym
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.figsize'] = [5,5]

In [ ]:
#Modification de la taille des sorties scrollables
%%html
<style>
.jp-CodeCell.jp-mod-outputsScrolled .jp-Cell-outputArea, div.output_scroll { max-height: 70vh; height:70vh;}
</style>

Nous pouvons à présent instancier un environnement :

In [ ]:
lava_env = gym.make("GridLava-v0")
lava_env.reset()
lava_env.render(mode="notebook")

L'agent démarre sur la case bleue. La case verte est l'objectif. Les cases gris fonçé représentent les murs. Les cases rouges représentent la lave.

Nous allons maintenant instancier un agent dans cet environnement, et afficher les valeurs attribuées aux états après son initialisation.

In [ ]:
agent=TP.create_agent_from_env(lava_env,
agent_class=TP.agents.gridworld.GridworldTabularValueAgent,
agent_args={'gamma':0.9},
value_class=TP.value_functions.tabular.TabularQValue,
value_args={'lr':0.5},
policy_class=TP.policies.greedy.GreedyQPolicy,
policy_args={}
)
agent.show_values(save_dir=None)

Les environnements *gridworld* présentent l'avantage de pouvoir représenter de façon superposée l'environnement et la fonction de valeur estimée par l'agent, grace au fait que chaque case de la grille correspond à un état de l'environnement. Pour l'instant, l'agent n'a pas encore commencé à explorer, son estimation de la fonction de valeur est donc 0 pour tous les états.

Nous allons entrainer l'agent pendant 200 épisodes en affichant à interval régulier sa fonction de valeur. Nous pouvons constater la propagation du signal de récompense depuis le but, jusqu'à la case de départ de l'agent, au cours de l'entrainement.

In [ ]:
agent.train(n_episodes=40,test_interval=5)

Nous pouvons également afficher les statistiques de l'entrainement :

In [ ]:
agent.plot_stats(save_dir=None)

On constate effectivement que l'agent a appris à se rendre à l'objectif par le chemin du haut (entre les cases de lave) et à obtenir le score maximal de 10 à chaque épisode. Dans le cas déterministe, ce *gridworld* est effectivement très simple à résoudre : $\pi_*(s) = East$ $\forall s \in \mathsf(S)$

L'ensemble des figures affichées ont été sauvegardées dans le dossier `results`, rangées dans un sous dossier correspondant à cette expérience. À l'intérieur de ce dossier, on trouve le fichier `infos.json` qui résume les paramètres de l'expérience, ainsi que les figures.

L'ensemble de ce cours utilisera des environnements compatibles [OpenAI-Gym](https://github.com/openai/gym), afin de profiter de ses interfaces simples et de son écosystème riche et uniformisé.

Vous êtes invités à prendre un moment pour consulter le dossier `code_docs`. Utilisez les boutons "*View Source*" pour lire les implémentations des classes `QLearningAgent` et `TabularQValue` utilisées dans l'expérience précédente. Nous enrichirons ensemble ce module au fil des séances et des exercices. Vous pouvez aussi utiliser la fonction `help` de python :

In [ ]:
help(TP.agents.gridworld.GridworldTabularValueAgent)

### Cas stochastique - Hyperparamètres

Nous allons maintenant introduire une source d'aléatoire dans la fonction de transition de l'environnement : à chaque pas de temps, il y a une probabilité $\tau$ pour que le déplacement aléatoire et indépendent de l'action choisie par l'agent : 

In [ ]:
stochastic_lava_env = TP.environments.gridworld_utils.StochasticWrapper( 
    gym.make("GridLava-v0") , 
    stochasticity=0.2
)

Instancions, dans cet environnement stochastique, un agent avec les mêmes paramètres que précédemment,et entrainons-le :

> **Exercice :** Avant de lancer l'entrainement dans la cellule ci-dessous, essayez de deviner l'impact de ce changement sur l'approximation de la fonction de valeur.

In [ ]:
stochastic_agent=TP.create_agent_from_env(stochastic_lava_env,
agent_class=TP.agents.gridworld.GridworldTabularValueAgent,
agent_args={'gamma':0.9},
value_class=TP.value_functions.tabular.TabularQValue,
value_args={'lr':0.1},
policy_class=TP.policies.greedy.GreedyQPolicy,
policy_args={}
)
stochastic_agent.train(n_episodes=1000,test_interval=100)
stochastic_agent.plot_stats(save_dir=None)

> **Exercice :** Étant donnée l'approximation de la fonction de valeur apprise par l'agent, vers quelle politique l'entrainement a-t-il convergé ? Comment expliquer ce comportement ? Comment empêcher, sans changer le *discount* $\gamma$, l'agent d'apprendre cette stratégie ?

**Répondre ici**

Jusqu'ici, nous avons utilisé uniquement la politique dite *greedy* (càd celle qui sélectionne toujours l'action de valeur maximale). Nous allons à présent utiliser la politique $\epsilon$-*greedy* ($0 \le \epsilon \le 1$) qui consiste à choisir une action aléatoire avec une probabilité $\epsilon$, et choisir l'action *greedy* le reste du temps. Lors des épisodes de test, l'action *greedy* est choisie 100% du temps ($\epsilon_{test} = 0$) :

In [ ]:
e_greedy_agent=TP.create_agent_from_env(stochastic_lava_env,
agent_class=TP.agents.gridworld.GridworldTabularValueAgent,
agent_args={'gamma':0.9},
value_class=TP.value_functions.tabular.TabularQValue,
value_args={'lr':0.1},
policy_class=TP.policies.greedy.EGreedyPolicy,
policy_args={'epsilon':0.33}
)
e_greedy_agent.train(n_episodes=2000,test_interval=100)
e_greedy_agent.plot_stats(save_dir=None)

> **Exercice :** Quelle est la politique apprise par ce nouvel agent ? Pouvez vous donner un exemple de politique qui donnerait de meilleurs retours ? En quoi l'aléatoire introduit par l'environnement stochastique diffère-t-il fondamentalement de l'aléatoire introduit par le politique $\epsilon$-greedy (en ce qui concerne l'apprentissage de l'agent) ?

**Répondre ici**

> **Exercice :** Dans la cellule ci-dessous, testez différentes valeurs pour les paramètres suivants :
>
> 1. du *taux de discount* (`gamma`) 
> 2. du *taux d'exploration* (`epsilon`) 
> 3. du *taux d'apprentissage* (`lr`)
>
> On introduit également les paramètres, `epsilon_decay` et `epsilon_min`. Consulter le code de la classe `EGreedyPolicy` pour comprendre leur fonctionnement. Le paramètre `default_value` permet de définir la valeur d'initialisation de $q[s][a]$ pour les paires état-action inconnues.
>
> Observez l'influence du choix de ces paramètres sur la fonction de valeur approximée, la politique *greedy* correspondante, et la vitesse de l'entrainement.
>
>Trouvez un ensemble de valeurs pour tous les *hyperparamètres* permettant d'apprendre la politique optimale en un minimum d'intérations.

In [ ]:
custom_agent=TP.create_agent_from_env(stochastic_lava_env,
agent_class=TP.agents.gridworld.GridworldTabularValueAgent,
agent_args={'gamma':0.9},
value_class=TP.value_functions.tabular.TabularQValue,
value_args={'lr':0.1, 'default_value': 0},
policy_class=TP.policies.greedy.EGreedyPolicy,
policy_args={'epsilon':0.33, 'epsilon_decay':0, 'epsilon_min':0.33}
)
custom_agent.train(n_episodes=2000,test_interval=100)
custom_agent.plot_stats(save_dir=None)